# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

## 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [13]:
# pipでオリジナルの解答に必要なライブラリーをインストール
!pip install --upgrade pip
!pip install -U pandas numpy scikit-learn imbalanced-learn

# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_category = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')
df_store = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv')
df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.4)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.19.4)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)
Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.7.0)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_P002=df_receipt[['sales_ymd','customer_id','product_cd','amount']]
df_P002.head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_P003=df_P002.rename(columns={"sales_ymd":"sales_date"})
df_P003.head(10)

#.head(10)と続けても表示されなかった（エラーにもならず）

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


.headと続けたら表示されなかった

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
#df_P004=df_P002.query('customer_id == "CS018205000001"')
#df_P004

#df_P004=df_receipt.copy() #Copyすることでオリジナルを汚さずチーム作業に向いてる
#df_P004

#df_P004=df_P004[['sales_ymd','customer_id','product_cd','amount']]
df_P004=df_P002[df_P002['customer_id']== "CS018205000001"]
df_P004
#df_P004[df_P004['customer_id']== "CS018205000001"]

#「'」だとNG

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
#df_P005=df_P004[df_P004['amount']>=1000]
#df_P005

df_P005=df_receipt.copy()
df_P005=df_P005[["sales_ymd","customer_id","product_cd","amount"]]
df_P005
df_P005[(df_P005["customer_id"] =="CS018205000001") & (df_P005["amount"] >=1000)]


#(df[df['X']>y]) , (df.query('X>y')) どちらも可  

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_P006=df_receipt.copy()
df_P006=df_P006[['sales_ymd','customer_id','product_cd','quantity','amount']]
df_P006[(df_P006['customer_id'] == 'CS018205000001') & ((df_P006['amount'] >= 1000) | (df_P006['quantity'] >=5))]

#P004にquantity追加＆条件指定はできるのか

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_P007=df_receipt.copy()
df_P007=df_P007[['sales_ymd','customer_id','product_cd','amount']]
df_P007[(df_P007['customer_id'] == 'CS018205000001') & 
        (df_P007['amount'] >=1000) & 
        (df_P007['amount'] <= 2000)]

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


df_P007=df_receipt.copy()
df_P007=df_P007[['sales_ymd','customer_id','product_cd','amount']]---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_P008=df_receipt.copy()
df_P008=df_P008[['sales_ymd','customer_id','product_cd','amount']]
df_P008[(df_P008['customer_id'] == 'CS018205000001') & (df_P008['product_cd'] != "P071401019")]

#df_P008=df_P004.query('product_cd != "P071401019"')
#df_P008

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:

df_store.query('not(prefecture_cd == "13" | floor_area>900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [ ]:
df_P009=df_store.copy()
df_P009
df_P009[(df_P009['prefecture_cd'] != 13) & (df_P009['floor_area'] <= 900)]
         

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0



---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_P010=df_store.copy()
df_P010[df_P010["store_cd"].str.find("S14") > -1].head(10)

#df_P010=df_store[df_store['store_cd'].str.startswith("S14")]
#df_P010.head(10)

#df_P010=df_store.query('store_cd.str.startswith("S14")')できない

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_P011=df_customer[df_customer['customer_id'].str.endswith("1")].head(10)
df_P011

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_P012=df_store.copy()
df_P012[df_P012["address"].str.find("横浜市") > -1]

#df_P012=df_store[df_store['address'].str.contains("横浜市")]
#df_P012

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_P013=df_customer.copy()
df_P013=df_P013[df_P013["status_cd"].str.contains('^[A-F]+.*')]
df_P013.head(10)

#^（ハット)一番前
#match一番初め^不要

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
#df_P014=df_customer[df_customer["status_cd"].str.match('.*[1-9]+$')].head(10)
#df_P014

df_P014=df_customer[df_customer["status_cd"].str.endswith(('1','2','3','4','5','6','7','8','9'))]
df_P014

#endswithで1-9まとめてはできない？

#こわぐちさん listに入れることで正規表現なしで書けるパターン
##str_list = [str(n) for n in list(range(1,10))]
#str_tuple = tuple(str_list)
##str_tuple

#df_customer[df_customer['status_cd'].str.endswith(str_tuple)]

#しんせいたろうさん
#df_customer[df_customer["status_cd"].str.contains('[1-9]$')]


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
...,...,...,...,...,...,...,...,...,...,...,...
21943,CS002515000242,加瀬 はるみ,1,女性,1966-06-23,52,184-0015,東京都小金井市貫井北町**********,S13002,20170102,A-20100124-7
21946,CS034412000010,末永 まひる,1,女性,1971-07-23,47,213-0035,神奈川県川崎市高津区向ケ丘**********,S14034,20150327,3-20090322-5
21952,CS002513000310,柳田 美嘉,1,女性,1963-09-17,55,187-0022,東京都小平市上水本町**********,S13002,20160911,1-20100711-2
21955,CS001413000156,上原 恵望子,1,女性,1974-12-10,44,144-0056,東京都大田区西六郷**********,S13001,20150724,1-20080210-1


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_P015=df_customer.copy()
df_P015=df_customer[(df_customer["status_cd"].str.match('[A-F]+.*'))&(df_customer["status_cd"].str.match('.*[1-9]+$'))].head(10)
df_P015

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_P016=df_store[df_store["tel_no"].str.match("[0-9]{3}-[0-9]{3}-[0-9]{4}$")]
df_P016

# $はここで終わり

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


In [ ]:
len(df_P016)

34

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_P017=df_customer.sort_values("birth_day").head(10)
df_P017

#本当はassending=True（小さい順・昇順）が必要？デフォルトにより省略可

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_P018=df_customer.sort_values("birth_day",ascending=False).head(10)
df_P018

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
#しんせいたろうさん回答. 
# method='min'とすると最小値が順位となる。1位、2位タイ、2位タイ、4位
df_P019 = df_receipt.copy()

df_P019["rank_amount"] = df_P019["amount"].rank(method='min', ascending=False)
df_P019.sort_values(by="rank_amount")[["customer_id", "amount","rank_amount"]].head(10)

#かめちゃんさん
#df_P019 = df_receipt.copy()
#df_P019["rank"] = df_receipt["amount"].rank(method="max", ascending=False)
#df_P019 = df_P019.sort_values("rank")
#df_P019.head(10)


,customer_id,amount,rank_amount
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_P020 = df_receipt.copy()
df_P020["rank_amount"] = df_P020["amount"].rank(method='first', ascending=False)
df_P020.sort_values(by="rank_amount")[["customer_id", "amount","rank_amount"]].head(10)

#first：同一値は登場順に順位付け。数値のみに有効。
#ascending=False:降順

,customer_id,amount,rank_amount
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
#unique重複ないデータを取りたい時
#len(df_receipt['customer_id'].unique())

#customer_idをgroupbyしてからもOK
#nunique

df_receipt["customer_id"].nunique()
#nを取るとデータそのものを返す。”n”をつけると数になる

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupby("store_cd")[["amount""quantity"]].sum()

#df_receipt.groupby("store_cd")だけでは同じstore_cdをグルーピングしただけで何も返してこない#そのあとの指示がいる
#list(df_receipt.groupby("store_cd"))
#list(df_receipt.groupby("store_cd"))[0]
#list(df_receipt.groupby("store_cd"))[0][0]
#list(df_receipt.groupby("store_cd"))[0][1]
#list(df_receipt.groupby("store_cd"))[0][1][["amount""quantity"]].sum()
#mean()
#first()
#pandasはSQLをモデルにしてる
#groupbyはリストでグルーピングするものを指定できる

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
#int64=数字→datetimeにする
df_P024=df_receipt.copy()
df_P024["sales_ymd"] =pd.to_datetime(df_P024["sales_ymd"],format="%Y%m%d")
df_P024.groupby("customer_id")["sales_ymd"].max().head(10)




customer_id
CS001113000004   2019-03-08
CS001114000005   2019-07-31
CS001115000010   2019-04-05
CS001205000004   2019-06-25
CS001205000006   2019-02-24
CS001211000025   2019-03-22
CS001212000027   2017-01-27
CS001212000031   2018-09-06
CS001212000046   2017-08-11
CS001212000070   2019-10-18
Name: sales_ymd, dtype: datetime64[ns]

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
#apply1つ１つのセルに対して実行する。
#024のmaxをminに置き換える

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [9]:
#max
#min
#max,minはシリーズ（s) dfではない
#merge
#rename
#concatの方が使いやすい。同じインデックスを持ってるとくっつく。 #marge

#s_P026=df_receipt.copy()
#s_P026_max=s_P026.groupby('customer_id').sales_ymd.max()
#s_P026_min=s_P026.groupby('customer_id').sales_ymd.min()
#s_P026_max.name="sales_ymd_max"
#s_P026_min.name="sales_ymd_min"
#s_P026_max_min=pd.concat([s_P026_max!=s_P026_min],axis=1)
#s_P026_max_min.head(10)
#↑うまくできなかった。。

#かめちゃんさん
df_receiptP026 = df_receipt.groupby("customer_id")[["sales_ymd"]].max()
df_receiptP026["sales_ymd_min"] = df_receipt.groupby("customer_id")[["sales_ymd"]].min()
df_receiptP026[df_receiptP026["sales_ymd"] != df_receiptP026["sales_ymd_min"]].head(10)

,sales_ymd,sales_ymd_min
customer_id,,
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [10]:
#df_P027=df_receipt.copy()
#df_P027=df_receipt.groupby("store_cd")["amount"].mean().sort_values(ascending=False)
#df_P027.head(5)
#こわぐちさん

df_receiptP027 = df_receipt.groupby("store_cd")[["amount"]].mean()
df_receiptP027.sort_values("amount", ascending=False).head()
#カメちゃんさん

,amount
store_cd,
S13052,402.867470
S13015,351.111960
S13003,350.915519
S14010,348.791262
S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [11]:
#こわぐちさん
df_P028=df_receipt.copy()
df_P028=df_receipt.groupby("store_cd")["amount"].median().sort_values(ascending=False)
df_P028.head(5)

#かめちゃんさん
#df_receiptP028 = df_receipt.groupby("store_cd")[["amount"]].median()
#df_receiptP028.sort_values("amount", ascending=False).head()

store_cd
S13052    190
S14010    188
S14050    185
S14040    180
S13003    180
Name: amount, dtype: int64

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [12]:
#mode
#groupbyで使われているメソッドは決まっている
#df_P029=df_receipt.copy()

df_receiptP029 = df_receipt.groupby("store_cd").product_cd.apply(lambda x: x.mode())
df_receiptP029.head()
#かめちゃんさん


store_cd   
S12007    0    P060303001
S12013    0    P060303001
S12014    0    P060303001
S12029    0    P060303001
S12030    0    P060303001
Name: product_cd, dtype: object

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [14]:
#分散　var
df_receiptP030 = df_receipt.groupby("store_cd")[["amount"]].var()
df_receiptP030.sort_values("amount", ascending=False).head()

#かめちゃんさん

,amount
store_cd,
S13052,441863.252526
S14011,306442.242432
S14034,297068.392740
S13001,295558.842618
S13015,295427.197086


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [15]:
#標準偏差　np.std
#groupby 同じ値を持つデータをまとめてそれぞれの塊に対して共通の操作を行いたいとき

df_receiptP031 = df_receipt.groupby("store_cd")[["amount"]].std()
df_receiptP031.sort_values("amount", ascending=False).head()

#カメちゃんさん

,amount
store_cd,
S13052,664.727954
S14011,553.572256
S14034,545.039808
S13001,543.653237
S13015,543.532149


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [16]:
#データを小さい順に並べた時、初めから数えて全体の100α%に位置する値を100αパーセンタイルという（0≦α≦1)。1/4分位数は25パーセンタイルと同じ。quantile()

df_receiptP032 = df_receipt["amount"].quantile([0, 0.25, 0.5, 0.75, 1.0])
df_receiptP032

#カメちゃんさん

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [17]:
df_receiptP033=df_receipt.copy()
df_receiptP033=df_receipt.groupby("store_cd")[["amount"]].mean()
df_receiptP033[df_receiptP033["amount"]>=330]

,amount
store_cd,
S12013,330.194130
S13001,348.470386
S13003,350.915519
S13004,330.943949
S13015,351.111960
S13019,330.208616
S13020,337.879932
S13052,402.867470
S14010,348.791262


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [23]:
df_receiptP034=df_receipt.copy()
df_receiptP034=df_receiptP034[~df_receiptP034["customer_id"].str.match('[Z]+.*')
df_receiptP034=df_receiptP034.reset_index()
df_receiptP034=df_receiptP034.groupby("customer_id")[["amount"]].sum()
df_receiptP034["amount"].mean()

#かめちゃんさん　３つに分けて実行
df_receiptP034= df_receipt.groupby("customer_id")[["amount"]].sum()
df_receiptP034.head()

df_receiptP034 = df_receiptP034.reset_index()
df_receiptP034 = df_receiptP034[~df_receiptP034["customer_id"].str.startswith("Z")]
df_receiptP034.head()

df_receiptP034["amount"].mean()

SyntaxError: ignored

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [24]:
#かめちゃんさん　
df_receiptP035= df_receipt.groupby("customer_id")[["amount"]].sum()
df_receiptP035 = df_receiptP035.reset_index()
df_receiptP035 = df_receiptP035[~df_receiptP035["customer_id"].str.startswith("Z")]
df_receiptP035.head()

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [25]:
#かめちゃんさん　
mean35 = df_receiptP035["amount"].mean()
mean35

2547.742234529256

In [26]:
df_receiptP035[df_receiptP035["amount"] > mean35].head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [27]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [28]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [32]:
df_receiptP036=pd.merge(df_receipt,df_store,on="store_cd")
df_receiptP036.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0


In [33]:
#かめちゃんさん
columns_list36 = list(df_receipt.columns.values) + ["store_name"]
columns_list36

['sales_ymd',
 'sales_epoch',
 'store_cd',
 'receipt_no',
 'receipt_sub_no',
 'customer_id',
 'product_cd',
 'quantity',
 'amount',
 'store_name']

In [35]:
df_receiptP036[columns_list36].head(10)

#ししさん
#pd.merge(df36_receipt,df36_store[["store_cd","store_name"]],on="store_cd").head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [36]:
#ししさん
df37_p=df_product.copy()
df37_c=df_category.copy()
df37_p.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [37]:
df37_c.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [38]:
pd.merge(df37_p,df37_c[["category_small_cd","category_small_name"]],on="category_small_cd").head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101002,4,401,40101,218.0,164.0,弁当類
2,P040101003,4,401,40101,230.0,173.0,弁当類
3,P040101004,4,401,40101,248.0,186.0,弁当類
4,P040101005,4,401,40101,268.0,201.0,弁当類
5,P040101006,4,401,40101,298.0,224.0,弁当類
6,P040101007,4,401,40101,338.0,254.0,弁当類
7,P040101008,4,401,40101,420.0,315.0,弁当類
8,P040101009,4,401,40101,498.0,374.0,弁当類
9,P040101010,4,401,40101,580.0,435.0,弁当類


In [ ]:
#かめちゃんさん　５つに分けて実行

df_product.columns

df_category.columns

df37 = pd.merge(df_product, df_category, on=["category_major_cd", "category_medium_cd", "category_small_cd"])
df37.head()

columns_list37 = list(df_product.columns.values) + ["category_small_name"]
columns_list37

df37[columns_list37].head(10)

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
#カメちゃんさん　５つに分けて実行

df_customer.head()

df_receipt[df_receipt["customer_id"].str.startswith("Z")].head()

df_receipt.head()

df38= pd.merge(df_receipt, df_customer, on="customer_id", how='left')
df38 = df38[(df38["gender_cd"] == 1) & (~df38["customer_id"].str.startswith("Z"))]
df38 = df38.fillna({"amount": 0})
df38.head()

df38_sum = df38.groupby("customer_id")[["amount"]].sum()
df38_sum.head(10)


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
#ししさん
df39_r = df_receipt.copy()

df39_day_cnt= df39_r[~df39_r["customer_id"].str.startswith("Z")].groupby("customer_id")["sales_ymd"].count().head(20)

df39_sales_amount = df39_r[~df39_r["customer_id"].str.startswith("Z")].groupby("customer_id")["amount"].sum().head(20)

pd.merge(df39_day_cnt,df39_sales_amount,how="outer",on="customer_id")

#カメちゃんさん
df39 = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]

df39_day = df39.groupby("customer_id")[["sales_ymd"]].count()
df39_day = df39_day.sort_values("sales_ymd", ascending=False).head(20)
df39_day.head()

df39_pay = df39.groupby("customer_id")[["amount"]].sum()
df39_pay = df39_pay.sort_values("amount", ascending=False).head(20)
df39_pay.head()

df39_merge = pd.merge(df39_day, df39_pay, on="customer_id", how="outer")
df39_merge = df39_merge.fillna({"amount":0})
df39_merge

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [39]:
len(df_product) * len(df_store)

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
#カメちゃんさん
df41 = df_receipt.groupby("sales_ymd")[["amount"]].sum()
df41 = df41.sort_index()
df41["yesterday"] = df41[["amount"]].shift(-1)
df41["change_amount"] = df41["yesterday"] - df41["amount"]
df41.head(10)

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
#かめちゃんさん
df42 = df_receipt.groupby("sales_ymd")[["amount"]].sum()
df42 = df42.sort_index()
df42["ago_1"] = df42[["amount"]].shift(-1)
df42["ago_2"] = df42[["amount"]].shift(-2)
df42["ago_3"] = df42[["amount"]].shift(-3)
df42.head(10)

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
#かめちゃんさん
df43 = pd.merge(df_receipt, df_customer, on="customer_id")
df43.head()

def get_generation(age):
  return int(age // 10) * 10

df43["generation"] = df43["age"].apply(get_generation)
df_sales_summary = df43.groupby(["gender", "generation"])[["amount"]].sum()
df_sales_summary = df_sales_summary.reset_index()
df_sales_summary.tail()

df43_ans = pd.crosstab(df_sales_summary["generation"], df_sales_summary["gender"] ,df_sales_summary["amount"], aggfunc="sum")
df43_ans

#ゆかりおさん
df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_sales_summary = pd.pivot_table(df_tmp, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
#かめちゃんさん
df44 = pd.merge(df_receipt, df_customer, on="customer_id")
df44["generation"] = df44["age"].apply(get_generation)
#df44.head()
df_sales_summary44 = df44.groupby(["gender_cd","gender", "generation"])[["amount"]].sum()
df_sales_summary44 = df_sales_summary44.reset_index()
df_sales_summary44.head()

#df44_ans = pd.crosstab(df_sales_summary44["gender"], [df_sales_summary44["gender_cd"], df_sales_summary44["generation"]] ,df_sales_summary44["amount"], aggfunc="sum")
df44_ans = pd.crosstab(df_sales_summary44["gender"], df_sales_summary44["generation"] ,df_sales_summary44["amount"], aggfunc="sum")
df44_ans

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
#かめちゃんさん
df_customer.head()

df45["birth_day"].head()

df45 = df_customer.copy()
df45["birth_day"] = pd.to_datetime(df45["birth_day"], format="%Y-%m-%d")
df45["birth_day"] = df45["birth_day"].dt.
df45.head()

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！